In [1]:
## Imports
import pandas as pd
import numpy as np
import time
import re

import spacy
from collections import Counter

np.random.seed(3000)

print("Imports completed")


Imports completed


In [2]:
LANGUAGE = 'ger'
SOURCE_DATASET = 'DEPlain'

OUTPUT_DATASET_SIZE = 5000
NUM_OF_CANDIDATE_SETS = 10

SHORT_OUTPUT_DATASET_SIZE = 200

filter_out_problematic_instances = True
clean_up = True
measure_syntactic_diversity = True
measure_lexical_diversity = True


execution_start_time = time.time()
execution_start_time_string = time.strftime("%m_%d_%Hh")

output_file_name = "data_subset_" + LANGUAGE + "_" + str(OUTPUT_DATASET_SIZE) + "_from_" + SOURCE_DATASET + "_" + execution_start_time_string
print("Output file name is going to be: ", output_file_name)

Output file name is going to be:  data_subset_ger_5000_from_DEPlain_09_05_17h


In [3]:
## Get the aligned sentences

source_file = '../../../datasets_original/DEPlain/E__Sentence-level_Corpus/DEplain-APA-sent/all.csv'
original_df = pd.read_csv(source_file)
original_df.head()


,original,simplification,original_id,simplification_id,pair_id,domain,corpus,language_level_original,language_level_simple,license,author,simple_url,complex_url,simple_title,complex_title,access_date,rater,alignment
0,"Am Mittwoch hat der Iran bekanntgegeben, dass ...",Der Iran wird teilweise aus dem Atom-Abkommen ...,296-1-1-0,296-0-1-0,296,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,1,1:1
1,"Die Präsidentin der EU-Kommission, Ursula von ...",Brüssel Ursula von der Leyen ist die Präsident...,628-1-1-0,628-0-1-0|628-0-1-1|628-0-1-2,628,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,1,1:3
2,So sollen die Treibhaus-Gase in der EU bis zum...,Bis zum Jahr 2030 soll es in der EU um 55 Proz...,628-1-1-2,628-0-1-4,628,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,1,1:1
3,In den letzten 29 Jahren schaffte man eine Ver...,"Das ist sehr viel, denn in den letzten 29 Jahr...",628-1-1-3,628-0-1-5,628,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,1,1:1
4,Für das neue Ziel bleiben weniger als 10 Jahre.,Für das neue Ziel bleiben aber weniger als 10 ...,628-1-1-4,628-0-1-6,628,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,1,1:1


In [4]:
my_column = original_df["alignment"]

percentage = my_column.value_counts(normalize=True) * 100
print(percentage.head(10))


1:1    75.537266
1:2    15.759793
2:1     2.766347
2:2     2.362445
1:3     1.836610
2:3     0.617284
1:4     0.312452
3:2     0.182899
2:4     0.167657
3:1     0.121933
Name: alignment, dtype: float64


In [5]:
relevant_df = original_df.copy()

In [6]:
if filter_out_problematic_instances:

    def has_multiple_sentences(text):
        # Match sentence-ending punctuation followed by a space and a capital letter (basic sentence structure)
        return bool(re.search(r'[.!?]\s+[A-ZÀ-Ý]', text))

    def contains_artifacts(text):   ## do not call for French
        # Define a regex pattern to match common artifact patterns
        # This pattern checks for unusual characters or sequences that are likely artifacts
        artifact_pattern = re.compile(r'[^\x20-\x7E]')  # Matches non-printable or unusual ASCII characters
        return bool(artifact_pattern.search(text))
            
    def contains_many_digits(text, threshold_base=5, threshold_increment=0.1):
        threshold = threshold_base + threshold_increment * len(text)
        digits = len(re.findall(r'\d', text))
        return digits > threshold
    
    def contains_many_special_chars(text, threshold_base=3, threshold_increment=0.1):
        threshold = threshold_base + threshold_increment * len(text)
        # Regex to match any character that is not a letter, digit, or common punctuation, including French accented characters
        special_chars = len(re.findall(r'[^a-zA-Z0-9\sÀ-ž]', text))
        return special_chars > threshold
    
    
    
    
    filtered_indices = []
    counter = 0
    sents_w_artifacts = 0
    sents_w_many_spec_chars = 0
    sents_w_many_digits = 0
    sents_that_are_multiple_sents = 0
    
    
    # Iterate over the DataFrame and find rows with artifacts
    for index, row in relevant_df.iterrows():
        sentence = row['original']
        counter += 1

        if has_multiple_sentences(sentence):
            sents_that_are_multiple_sents += 1
            filtered_indices.append(index)
            if counter % 30 == 0:
                print("|| Found multiple sentences in instance " + str(index) + " ||")
                print(sentence)

        elif contains_many_digits(sentence):
            sents_w_many_digits += 1
            filtered_indices.append(index)
            if counter % 20 == 0:
                print("|| Too many digits in instance " + str(index) + " ||")
                print(sentence)

            
        else:
            if contains_many_special_chars(sentence):
                sents_w_many_spec_chars += 1
                filtered_indices.append(index)
                if counter % 10 == 0:
                    print("|| Too many special characters in instance " + str(index) + " ||")
                    print(sentence)         
            
            
    
    # Create a new DataFrame excluding the rows with artifacts
    clean_df = relevant_df.drop(index=filtered_indices)
    df = clean_df
    
    print("Filtering complete")
    #print("Filtered out ", sents_w_artifacts, " sentences containing artifacts.")
    print("Filtered out ", sents_that_are_multiple_sents, " instances with multiple sentences.")
    print("Then filtered out ", sents_w_many_digits, " sentences containing many digits.")
    print("Then filtered out ", sents_w_many_spec_chars, " sentences containing too many special characters.")



|| Found multiple sentences in instance 149 ||
Manches wurde aber auch billiger. Am stärksten sank der Preis bei Kaffee. Auch Bekleidung und Schuhe wurden billiger.
|| Found multiple sentences in instance 299 ||
Mutter und Kind geht es sehr gut, erzählte Prinz Harry. Er war bei der Geburt dabei.
|| Found multiple sentences in instance 449 ||
Am Dienstag haben Sänger von 17 Ländern im 1. Halbfinale von dem Musik-Wettbewerb gesungen.
|| Found multiple sentences in instance 809 ||
Im Jahr 1989 kam es in China zu großen Demonstrationen. Hunderttausende Menschen demonstrierten wochenlang auf dem Tiananmen-Platz in Peking.
|| Found multiple sentences in instance 1229 ||
Der wärmste See ist der Stubenbergsee in der Steiermark. Er ist 27 Grad warm.
|| Found multiple sentences in instance 1439 ||
Am Montag ist Kyriakos Mitsotakis als griechischer Minister-Präsident vereidigt worden. Mitsotakis ist der Anführer der Partei Nea Dimokratia.
|| Found multiple sentences in instance 1529 ||
Weil es in

In [7]:
print(len(relevant_df))
print(len(clean_df))
print(len(relevant_df)-len(clean_df))
print(len(clean_df)/len(relevant_df))

13122
11944
1178
0.910227099527511


In [8]:
if clean_up:

    def replace_LRB_RRB(text):
        # Replace "-LRB-" with "(" and "-RRB-" with ")"
        text = text.replace('-LRB- ', '(')
        text = text.replace(' -RRB-', ')')
        return text

    def replace_wrong_quotation_marks(text):
        # Replace patterns like `` word(s) '' with "word(s)"
        return re.sub(r'``\s*(.*?)\s*\'\'', r'"\1"', text)
    
    def remove_spaces_before_punctuation_FRENCH(text):
        # Regex pattern to match a space between a word-like string and a period or comma.
        return re.sub(r'(\w)\s+([.,\'])', r'\1\2', text)


    df['wiki_sent'] = df['original'].apply(replace_LRB_RRB)
    #df['wiki_sent'] = df['wiki_sent'].apply(replace_wrong_quotation_marks)
    df['wiki_sent'] = df['original'].apply(remove_spaces_before_punctuation_FRENCH)
    
    print("Clean up performed.")

else:
    print("No clean up")


Clean up performed.


In [9]:
counter = 0
for sentence in df['original']:
    counter += 1
    if counter % 100 == 0:
        print(sentence)

Winterreifen-Pflicht ab Donnerstag vorüber.
Seitdem gab es jedes Jahr mehr als 500.000 Anzeigen.
Dieses Geld nennt man Mindestsicherung.
Der Iran durfte zum Beispiel nicht mehr so viel Öl an andere Länder verkaufen wie zuvor.
Am Montag ist der ehemalige österreichische Formel-1-Rennfahrer Niki Lauda gestorben.
Schüler protestierten wieder für mehr Klima-Schutz.
Auf dem Ausflugsboot waren viele Urlauber aus Südkorea.
An diesem Tag landeten alliierte Truppen in der Normandie.
Davon waren mehr als 43.000 Buben und mehr als 41.000 Mädchen.
In China gibt es eine Gegend, die Sichuan heißt.
Und ein weiterer Orang-Utan wirft die Wanne so oft um, bis er komplett nass ist.
Die Polizei geht davon aus, dass der Mann wahrscheinlich mit einem Ast tödlich am Kopf verletzt wurde.
Sein Rekord liegt sogar bei 74 Hotdogs.
Dabei sterben immer wieder Menschen, zum Beispiel weil ihre Boote untergehen.
Die Komodo-Warane sollen einfach zu Fuß von ihrem alten Gehege übersiedeln.
Jetzt hat das Mädchen auch eine

In [10]:
def get_dependency_tree(doc):
    # Get the dependency structure in terms of (head, relation, dependent)
    return [(token.head.dep_, token.dep_, token.dep_) for token in doc if token.head != token]

def calculate_tree_diversity(parse_trees):
    # Flatten the list of trees and count unique structures
    flattened_trees = [tuple(tree) for trees in parse_trees for tree in trees]
    tree_counter = Counter(flattened_trees)
    
    # Diversity score: higher score means more unique structures
    diversity_score = len(tree_counter) / sum(tree_counter.values())
    
    return diversity_score

In [11]:
# Load the German model
nlp = spacy.load("de_core_news_lg")

candidate_sets = [df["original"].sample(n=OUTPUT_DATASET_SIZE, replace=False) for _ in range(NUM_OF_CANDIDATE_SETS)]

sets_w_scores = []

start_time_all = time.time()

for idx, candidate_set in enumerate(candidate_sets):

    start_time = time.time()

    print("Assessing set ", idx+1, "/", NUM_OF_CANDIDATE_SETS, " of len ", OUTPUT_DATASET_SIZE)

    # List of sentences
    sentences = candidate_set

    print(type(sentences))

    # Parse the sentences
    docs = [nlp(sentence) for sentence in sentences]

    print(len(docs))

    tree_diversity_score = None
    std_clause_density_avg = None
    std_token_prob_avg = None



    if measure_syntactic_diversity:
        # Extract parse trees for all sentences
        parse_trees = [get_dependency_tree(doc) for doc in docs]
    
        elapsed_time_trees = time.time() - start_time
        print("Done with the parse_trees after ", time.strftime("%H:%M:%S", time.gmtime(elapsed_time_trees)) + f".{int((elapsed_time_trees % 1) * 1000):03d}")
    
        # Calculate syntactic diversity score
        
        tree_diversity_score = calculate_tree_diversity(parse_trees)

    
        clause_density_avg_list = []
        for doc in docs:
            num_tokens = len([token for token in doc if token.is_alpha])
            num_clauses = sum(1 for token in doc if token.dep_ in ('ROOT', 'csubj', 'csubjpass', 'advcl', 'relcl', 'xcomp', 'ccomp'))
            clause_density = num_clauses / num_tokens if num_tokens else 0
            clause_density_avg_list.append(clause_density)
        
        std_clause_density_avg = np.std(clause_density_avg_list)

    if measure_lexical_diversity:
        token_prob_avg_list = []
        for doc in docs:
            # Calculate lexical diversity
            token_freqs = [token.prob for token in doc if token.is_alpha]  # Use token.prob as a proxy for frequency score
            token_prob_avg = np.mean(token_freqs) if token_freqs else 0
            token_prob_avg_list.append(token_prob_avg)
        
        std_token_prob_avg = np.std(token_prob_avg_list)
        
    
    sets_w_scores.append((idx, tree_diversity_score, std_clause_density_avg, std_token_prob_avg))
    
    elapsed_time_set = time.time() - start_time
    print("Finished set ",idx, "after ", time.strftime("%H:%M:%S", time.gmtime(elapsed_time_set)) + f".{int((elapsed_time_set % 1) * 1000):03d}")

elapsed_time_all = time.time() - start_time_all
print("Finished all after ", time.strftime("%H:%M:%S", time.gmtime(elapsed_time_all)) + f".{int((elapsed_time_all % 1) * 1000):03d}")
print(sets_w_scores)

Assessing set  1 / 10  of len  5000
<class 'pandas.core.series.Series'>
5000
Done with the parse_trees after  00:00:35.060
Finished set  0 after  00:00:35.265
Assessing set  2 / 10  of len  5000
<class 'pandas.core.series.Series'>
5000
Done with the parse_trees after  00:00:36.310
Finished set  1 after  00:00:36.547
Assessing set  3 / 10  of len  5000
<class 'pandas.core.series.Series'>
5000
Done with the parse_trees after  00:00:44.829
Finished set  2 after  00:00:45.105
Assessing set  4 / 10  of len  5000
<class 'pandas.core.series.Series'>
5000
Done with the parse_trees after  00:00:37.829
Finished set  3 after  00:00:38.094
Assessing set  5 / 10  of len  5000
<class 'pandas.core.series.Series'>
5000
Done with the parse_trees after  00:00:42.095
Finished set  4 after  00:00:42.330
Assessing set  6 / 10  of len  5000
<class 'pandas.core.series.Series'>
5000
Done with the parse_trees after  00:00:36.828
Finished set  5 after  00:00:37.140
Assessing set  7 / 10  of len  5000
<class 'pa

In [17]:
# Weights
weights = [0.3, 0.3, 0.4]

# Function to calculate the weighted sum for a tuple
def weighted_sum(tup):
    return weights[0] * tup[1] + weights[1] * tup[2] + weights[2] * tup[3]

# Find the tuple with the maximum weighted sum
best_tuple = max(sets_w_scores, key=weighted_sum)

print(sets_w_scores)
print(best_tuple)

[(0, 0.006139466691102355, 0.06531860268535505, 0.0), (1, 0.006128164096595869, 0.0628229167878242, 0.0), (2, 0.00625527387460102, 0.06040390156658362, 0.0), (3, 0.006271827972718463, 0.06016376294800875, 0.0), (4, 0.0061021284370147075, 0.061229153996163556, 0.0), (5, 0.005992413274936319, 0.06092302118970018, 0.0), (6, 0.006057519145762465, 0.06386254888542288, 0.0), (7, 0.006089978054133138, 0.06053383551586649, 0.0), (8, 0.006053180205552101, 0.057868895723378035, 0.0), (9, 0.00612143605580985, 0.05939437063894369, 0.0)]
(0, 0.006139466691102355, 0.06531860268535505, 0.0)


In [18]:
best_set_of_sentences = candidate_sets[best_tuple[0]]
print(best_set_of_sentences)

9412              Der Mann wurde dann ins Spital gebracht.
5185     Mit der 1-2-3-Karte soll man in Zukunft für al...
10329    Zusammen mit dem Friedens-Nobelpreis bekommt d...
1178          Boris Johnson droht mit einem harten Brexit.
11229    Der Käufer will für die Öffentlichkeit unerkan...
                               ...                        
659        Strache zeigt 3 Personen wegen Ibiza-Videos an.
6951     In öffentlichen Verkehrsmitteln und in Taxis g...
10462    Die Autofahrer sollen dann also nur noch 30 km...
575      Die Regierung von Österreich hat nach der Regi...
12313    Sie sind damit vermutlich besser vor einer ern...
Name: original, Length: 5000, dtype: object


In [19]:
with open(output_file_name, "w") as file:
    counter = 0
    for index, sentence in best_set_of_sentences.items():
        counter += 1
        file.write(f"{index}|{sentence}\n\n")
        if counter % 50 == 0:
            print(f"{index}|{sentence}\n\n")
        

5430|49 Prozent der befragten US-Bürger sind zufrieden mit seiner Arbeit als Präsident.


5457|Danach brach das Flugzeug in mehrere Teile und geriet teilweise in Brand.


4827|Nun gibt es einen neuen Generalsekretär.


8683|Dritter wurde überraschend der McLaren-Fahrer Lando Norris.


6243|Das heißt, er kann bei Menschen oder Tieren Krankheiten auslösen.


8953|Auch das war laut VfGH gesetzwidrig, weil nicht alle Geschäfte gleich behandelt wurden.


5044|Beim Welt-Wirtschafts-Forum treffen sich Politiker, Firmen-Chefs und Wissenschafter aus der ganzen Welt und sprechen über verschiedene Themen.


6397|Es gab bis Freitag zwar 39 neu angesteckte Menschen.


3806|Am Sonntag hatten schon die Grünen entschieden, dass sie mit der ÖVP über eine Regierungs-Bildung verhandeln wollen.


12121|Daher fordert die Arbeiterkammer eine Soforthilfe von 200 Euro pro Schulkind zur Familienbeihilfe.


2538|Für sie werden extra Rampen gebaut.


3334|Nun hat der Verfassungs-Gerichtshof entschieden, die Klag

In [25]:
short_output_file_name = "data_subset_" + LANGUAGE + "_" + str(SHORT_OUTPUT_DATASET_SIZE) + "_from_" + SOURCE_DATASET + "_" + execution_start_time_string

short_sample = best_set_of_sentences.sample(n=SHORT_OUTPUT_DATASET_SIZE, replace=False)

with open(short_output_file_name, "w") as file:
    counter = 0
    for index, sentence in short_sample.items():
        counter += 1
        file.write(f"{index}|{sentence}\n\n")
        if counter % 100 == 0:
            print(f"{index}|{sentence}\n\n")


7584|Mit Kleinstaaterei meint Van der Bellen, dass sich jedes EU-Land nur um sich selbst kümmert und keine Rücksicht auf die anderen Länder nimmt.


8596|Das österreichische Außenministerium hat eine Reisewarnung für die Länder des Westbalkans ausgesprochen.




In [26]:
execution_elapsed_time = time.time() - execution_start_time
print("Code execution completed after ", time.strftime("%H:%M:%S", time.gmtime(execution_elapsed_time)) + f".{int((execution_elapsed_time % 1) * 1000):03d}")

Code execution completed after  00:07:36.640
